In [1]:
# !nvidia-smi

In [2]:
gpu=3
CUDA_VISIBLE_DEVICES=gpu

In [3]:
import numpy as np
import pandas as pd

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler

import matplotlib.pyplot as plt
import time
import copy
from random import shuffle

import tqdm.notebook as tqdm

import sklearn
from sklearn.metrics import accuracy_score, cohen_kappa_score
from sklearn.metrics import classification_report
from PIL import Image
import cv2

# import osa
import shutil

import seaborn as sns
import os
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torchvision
from sklearn.metrics import roc_curve,auc, precision_score,precision_recall_curve,recall_score,precision_recall_fscore_support,confusion_matrix
import numpy as np
from prettytable import PrettyTable
print(torch.cuda.is_available())
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
print(torch.cuda.get_device_properties(0).total_memory)
print(torch.cuda.memory_allocated())
gpu_id = gpu

True
11721506816
0


In [4]:
df=pd.read_csv('/DATA/chowdari1/DATA/dataset/stanford/CheXpert-v1.0-small/train.csv')
test=pd.read_csv('/DATA/chowdari1/DATA/dataset/stanford/CheXpert-v1.0-small/valid.csv')
new_df=pd.read_csv("/DATA/chowdari1/DATA/csv/stanford_train.csv")

In [5]:
df.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN
2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [6]:
df.shape

(223414, 19)

In [7]:
df.isnull().values.sum()

2310361

In [8]:
df["No Finding"].isnull().values.any()
df["No Finding"].isnull().values.sum()

201033

In [9]:
df=df.fillna(-2)

In [10]:
df.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,0.0,-2.0,-2.0,-2.0,1.0
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,-2.0,-2.0,-1.0,1.0,-2.0,-1.0,-1.0,-2.0,-1.0,-2.0,-1.0,-2.0,1.0,-2.0
2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,-2.0,-2.0,-2.0,1.0,-2.0,-2.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,1.0,-2.0
3,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Lateral,-2,-2.0,-2.0,-2.0,1.0,-2.0,-2.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,1.0,-2.0
4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,-2.0,-2.0,-2.0,-2.0,-2.0,1.0,-2.0,-2.0,-2.0,0.0,-2.0,-2.0,-2.0,-2.0


In [11]:
s="No Finding"
df[s].unique()

array([ 1., -2.])

In [10]:
df.columns

Index(['Path', 'Sex', 'Age', 'Frontal/Lateral', 'AP/PA', 'No Finding',
       'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity',
       'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
       'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture',
       'Support Devices'],
      dtype='object')

In [12]:
count=0
list=[]
for i in range(df.shape[0]):
    s=""
    for j in df.columns[6:]:
        if df[j][i]==1:
            if(s!='' and (j=='Fracture' or j=='Support Devices')):
                s=s
            else:
                s=s+j
    if( s=='Support Devices' or s=='Fracture' or s=='FractureSupport Devices'):
        s='s/f'
    list.append(s)
len(list),count

(223414, 0)

In [13]:
len(list)

223414

In [14]:
df["label"]=list

In [15]:
count=0
for i in range(df.shape[0]):
    if(df['No Finding'][i]==1 and df['label'][i]=='') :
        count+=1
count

13573

In [16]:
count=0
for i in range(df.shape[0]):
    if(df['No Finding'][i]==1):
        count+=1
    elif(df['label'][i]=='s/f' and df['No Finding'][i]!=1):
        count+=1
count

31278

In [17]:
list

['s/f',
 'Lung Opacity',
 'Lung Opacity',
 'Lung Opacity',
 'Edema',
 '',
 '',
 's/f',
 's/f',
 'Pneumothorax',
 'Pneumothorax',
 '',
 'CardiomegalyLung OpacityAtelectasisPneumothorax',
 'Enlarged CardiomediastinumLung OpacityAtelectasis',
 'Lung OpacityPleural Effusion',
 'Lung OpacityPleural Effusion',
 'CardiomegalyAtelectasis',
 'CardiomegalyAtelectasis',
 '',
 '',
 '',
 '',
 'Lung Opacity',
 'Lung OpacityPneumothorax',
 'Lung Opacity',
 'Lung OpacityPneumothorax',
 'Lung OpacityConsolidationPleural Effusion',
 'Lung OpacityEdemaPleural Effusion',
 'Enlarged CardiomediastinumPleural Effusion',
 'Lung OpacityEdemaPleural Effusion',
 'Lung Opacity',
 'CardiomegalyEdemaAtelectasisPleural Effusion',
 'Lung OpacityPneumothorax',
 'Lung OpacityPneumothorax',
 'Pleural Effusion',
 'Pleural Effusion',
 's/f',
 's/f',
 's/f',
 's/f',
 '',
 '',
 '',
 '',
 'Lung OpacityEdema',
 'Edema',
 'Lung OpacityConsolidationPleural Other',
 'Lung OpacityConsolidationPleural Other',
 's/f',
 'Edema',
 'L

In [18]:
list
count=0
for i in list:
    if(i=='s/f'):
        count+=1
count, len(list)

(17705, 223414)

In [19]:
df['No Finding'].value_counts()

-2.0    201033
 1.0     22381
Name: No Finding, dtype: int64

In [20]:
df['label'].value_counts()[1:]

Lung Opacity                                                                                    21520
Lung OpacityPleural Effusion                                                                    20965
s/f                                                                                             17705
Pleural Effusion                                                                                 9639
Lung OpacityEdemaPleural Effusion                                                                8975
                                                                                                ...  
CardiomegalyLung OpacityEdemaConsolidationPneumothorax                                              1
Enlarged CardiomediastinumLung OpacityLung LesionConsolidationPneumoniaPleural Effusion             1
Enlarged CardiomediastinumLung LesionConsolidationPneumoniaPleural Effusion                         1
CardiomegalyLung OpacityLung LesionEdemaConsolidationAtelectasisPneumothorax      

In [21]:
df.shape

(223414, 20)

In [22]:
df['tar']=np.empty(df.shape[0])

In [23]:
cnan=0
count0=0
count1=0
val=0
etc=0
for i in range(df.shape[0]):
    s= df['label'][i]
    if(df['No Finding'][i]==1 or s=='s/f'):
        count0+=1
        val=0
    elif(s=='' and df['No Finding'][i]!=1):
        cnan+=1
        val=-2
    elif(s!='' or s!='s/f'):
        val=1
        count1+=1
    else:
        etc+=1
    df['tar'][i]=val
count0,count1,cnan,etc


(31278, 180995, 11141, 0)

In [197]:
df['tar'].value_counts()

 1.0    180995
 0.0     31278
-2.0     11141
Name: tar, dtype: int64

In [198]:
df.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,...,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,label,tar
0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,-2.0,-2.0,-2.0,-2.0,...,-2.0,-2.0,-2.0,0.0,-2.0,-2.0,-2.0,1.0,s/f,0.0
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,-2.0,-2.0,-1.0,1.0,-2.0,...,-1.0,-2.0,-1.0,-2.0,-1.0,-2.0,1.0,-2.0,Lung Opacity,1.0
2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,-2.0,-2.0,-2.0,1.0,-2.0,...,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,1.0,-2.0,Lung Opacity,1.0
3,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Lateral,-2,-2.0,-2.0,-2.0,1.0,-2.0,...,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,1.0,-2.0,Lung Opacity,1.0
4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,-2.0,-2.0,-2.0,-2.0,-2.0,...,-2.0,-2.0,-2.0,0.0,-2.0,-2.0,-2.0,-2.0,Edema,1.0


In [199]:
df["No Finding"].value_counts()

-2.0    201033
 1.0     22381
Name: No Finding, dtype: int64

In [200]:
x=df["label"].value_counts()[1:]
x

Lung Opacity                                                                                    21520
Lung OpacityPleural Effusion                                                                    20965
s/f                                                                                             17705
Pleural Effusion                                                                                 9639
Lung OpacityEdemaPleural Effusion                                                                8975
                                                                                                ...  
CardiomegalyLung OpacityEdemaConsolidationPneumothorax                                              1
Enlarged CardiomediastinumLung OpacityLung LesionConsolidationPneumoniaPleural Effusion             1
Enlarged CardiomediastinumLung LesionConsolidationPneumoniaPleural Effusion                         1
CardiomegalyLung OpacityLung LesionEdemaConsolidationAtelectasisPneumothorax      

In [201]:
count=0
for i in x:
    if(i<250):
        count+=i
    else :
        count+=250
count

33060

In [202]:
x.keys()

Index(['Lung Opacity', 'Lung OpacityPleural Effusion', 's/f',
       'Pleural Effusion', 'Lung OpacityEdemaPleural Effusion', 'Edema',
       'Lung OpacityEdema', 'Pneumothorax', 'Atelectasis',
       'EdemaPleural Effusion',
       ...
       'Lung OpacityLung LesionEdemaPneumoniaAtelectasis',
       'Lung LesionEdemaPneumoniaAtelectasis',
       'CardiomegalyLung OpacityLung LesionAtelectasisPneumothoraxPleural Effusion',
       'Enlarged CardiomediastinumCardiomegalyLung LesionPneumothorax',
       'Enlarged CardiomediastinumCardiomegalyPneumothoraxPleural Other',
       'CardiomegalyLung OpacityEdemaConsolidationPneumothorax',
       'Enlarged CardiomediastinumLung OpacityLung LesionConsolidationPneumoniaPleural Effusion',
       'Enlarged CardiomediastinumLung LesionConsolidationPneumoniaPleural Effusion',
       'CardiomegalyLung OpacityLung LesionEdemaConsolidationAtelectasisPneumothorax',
       'Enlarged CardiomediastinumLung OpacityEdemaConsolidationPneumoniaAtelectasisPleura

In [55]:
sum=0
for i in x[180:]:
    sum+=i
sum

15153

In [203]:
x

Lung Opacity                                                                                    21520
Lung OpacityPleural Effusion                                                                    20965
s/f                                                                                             17705
Pleural Effusion                                                                                 9639
Lung OpacityEdemaPleural Effusion                                                                8975
                                                                                                ...  
CardiomegalyLung OpacityEdemaConsolidationPneumothorax                                              1
Enlarged CardiomediastinumLung OpacityLung LesionConsolidationPneumoniaPleural Effusion             1
Enlarged CardiomediastinumLung LesionConsolidationPneumoniaPleural Effusion                         1
CardiomegalyLung OpacityLung LesionEdemaConsolidationAtelectasisPneumothorax      

In [204]:
dict=x.to_dict()
sum=0
for i in dict.keys():
    if(dict[i]>250):
        dict[i]=250
    sum+=dict[i]
sum

33060

In [205]:
dict

{'Lung Opacity': 250,
 'Lung OpacityPleural Effusion': 250,
 's/f': 250,
 'Pleural Effusion': 250,
 'Lung OpacityEdemaPleural Effusion': 250,
 'Edema': 250,
 'Lung OpacityEdema': 250,
 'Pneumothorax': 250,
 'Atelectasis': 250,
 'EdemaPleural Effusion': 250,
 'AtelectasisPleural Effusion': 250,
 'Cardiomegaly': 250,
 'Lung OpacityAtelectasis': 250,
 'Lung OpacityAtelectasisPleural Effusion': 250,
 'Lung OpacityPneumothorax': 250,
 'CardiomegalyLung OpacityEdemaPleural Effusion': 250,
 'Lung OpacityPneumothoraxPleural Effusion': 250,
 'CardiomegalyEdema': 250,
 'CardiomegalyLung OpacityPleural Effusion': 250,
 'Consolidation': 250,
 'CardiomegalyLung Opacity': 250,
 'Enlarged Cardiomediastinum': 250,
 'EdemaAtelectasisPleural Effusion': 250,
 'ConsolidationPleural Effusion': 250,
 'CardiomegalyLung OpacityEdema': 250,
 'Lung OpacityLung Lesion': 250,
 'EdemaAtelectasis': 250,
 'Lung Lesion': 250,
 'CardiomegalyEdemaPleural Effusion': 250,
 'PneumothoraxPleural Effusion': 250,
 'Lung Opac

In [110]:
df.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,...,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,label,tar
0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,-2.0,-2.0,-2.0,-2.0,...,-2.0,-2.0,-2.0,0.0,-2.0,-2.0,-2.0,1.0,s/f,0.0
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,-2.0,-2.0,-1.0,1.0,-2.0,...,-1.0,-2.0,-1.0,-2.0,-1.0,-2.0,1.0,-2.0,Lung Opacity,1.0
2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,-2.0,-2.0,-2.0,1.0,-2.0,...,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,1.0,-2.0,Lung Opacity,1.0
3,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Lateral,-2,-2.0,-2.0,-2.0,1.0,-2.0,...,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,1.0,-2.0,Lung Opacity,1.0
4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,-2.0,-2.0,-2.0,-2.0,-2.0,...,-2.0,-2.0,-2.0,0.0,-2.0,-2.0,-2.0,-2.0,Edema,1.0


In [206]:
df=df.sample(frac=1)
mana_df=pd.DataFrame()
size=64338
mana_df["Path"]=np.empty(size,dtype=object)
mana_df["target"]=np.empty(size,dtype=int)
mana_df["label"]=np.empty(size,dtype=object)

In [139]:
count0=0
count1=0
for i in range(df.shape[0]):
    s= df['label'][i]
    if(df['tar'][i]==0):
        count0+=1
    elif(df['tar'][i]==1):
        count1+=1
count0,count1

(31278, 180995)

In [136]:
dict

{'Lung Opacity': 250,
 'Lung OpacityPleural Effusion': 250,
 's/f': 250,
 'Pleural Effusion': 250,
 'Lung OpacityEdemaPleural Effusion': 250,
 'Edema': 250,
 'Lung OpacityEdema': 250,
 'Pneumothorax': 250,
 'Atelectasis': 250,
 'EdemaPleural Effusion': 250,
 'AtelectasisPleural Effusion': 250,
 'Cardiomegaly': 250,
 'Lung OpacityAtelectasis': 250,
 'Lung OpacityAtelectasisPleural Effusion': 250,
 'Lung OpacityPneumothorax': 250,
 'CardiomegalyLung OpacityEdemaPleural Effusion': 250,
 'Lung OpacityPneumothoraxPleural Effusion': 250,
 'CardiomegalyEdema': 250,
 'CardiomegalyLung OpacityPleural Effusion': 250,
 'Consolidation': 250,
 'CardiomegalyLung Opacity': 250,
 'Enlarged Cardiomediastinum': 250,
 'EdemaAtelectasisPleural Effusion': 250,
 'ConsolidationPleural Effusion': 250,
 'CardiomegalyLung OpacityEdema': 250,
 'Lung OpacityLung Lesion': 250,
 'EdemaAtelectasis': 250,
 'Lung Lesion': 250,
 'CardiomegalyEdemaPleural Effusion': 250,
 'PneumothoraxPleural Effusion': 250,
 'Lung Opac

In [207]:
count=0
class0=0
class1=0
other=0
for i in range(df.shape[0]):
    s= df['label'][i]
    if(df['tar'][i]==0):
        class0+=1
        mana_df["Path"][count]=df["Path"][i]
        mana_df["target"][count]=0
        mana_df["label"][count]=df["label"][i]
        count+=1
    else:
        if(df['tar'][i]==1 and dict[df["label"][i]]>0):            
            mana_df["Path"][count]=df["Path"][i]
            mana_df["target"][count]=1
            mana_df["label"][count]=df["label"][i]
            dict[df["label"][i]]+=-1
            count+=1
            class1+=1
        else:
            other+=1
    if(count==size):
        break
    
count, class0, class1, other

(64088, 31278, 32810, 159326)

In [208]:
print(size)

64338


In [209]:
mana_df.head()

,Path,target,label
0,CheXpert-v1.0-small/train/patient00001/study1/...,0,s/f
1,CheXpert-v1.0-small/train/patient00002/study2/...,1,Lung Opacity
2,CheXpert-v1.0-small/train/patient00002/study1/...,1,Lung Opacity
3,CheXpert-v1.0-small/train/patient00002/study1/...,1,Lung Opacity
4,CheXpert-v1.0-small/train/patient00003/study1/...,1,Edema


In [210]:
for i in range(mana_df.shape[0]):
    if(mana_df['target'][i]!=1 and mana_df['target'][i]!=0):
        mana_df=mana_df.drop(i)

In [211]:
mana_df['target'].value_counts()

1    32810
0    31278
Name: target, dtype: int64

In [213]:
mana_df.shape

(64088, 3)

In [171]:
test=pd.read_csv('/DATA/chowdari1/DATA/dataset/stanford/CheXpert-v1.0-small/valid.csv')

In [178]:
test.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0-small/valid/patient64541/study1/...,Male,73,Frontal,AP,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,CheXpert-v1.0-small/valid/patient64542/study1/...,Male,70,Frontal,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,CheXpert-v1.0-small/valid/patient64542/study1/...,Male,70,Lateral,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,CheXpert-v1.0-small/valid/patient64545/study1/...,Female,55,Frontal,AP,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
5,CheXpert-v1.0-small/valid/patient64546/study1/...,Male,56,Frontal,AP,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [175]:
test = test.reset_index(drop=True)

In [177]:
test=test.drop(3)

In [146]:
mana_df['target'].value_counts()

1          32810
0          31278
8521360      250
Name: target, dtype: int64

In [143]:
df["No Finding"].isnull().values.any()

False

In [214]:
mana_df=mana_df.drop(["label"],axis=1)

In [219]:
mana_df.head()

,target,path
0,0,CheXpert-v1.0-small/train/patient00001/study1/...
1,1,CheXpert-v1.0-small/train/patient00002/study2/...
2,1,CheXpert-v1.0-small/train/patient00002/study1/...
3,1,CheXpert-v1.0-small/train/patient00002/study1/...
4,1,CheXpert-v1.0-small/train/patient00003/study1/...


In [218]:
# mana_df["path"]=mana_df["Path"]
mana_df=mana_df.drop(["Path"],axis=1)

In [220]:
path="/DATA/chowdari1/DATA/csv/"
mana_df.to_csv(path+"mana_great_all_stanford.csv")

In [232]:
mana_df=mana_df.sample(frac=1)

In [233]:
mana_df.iloc[:28500]['target'].value_counts()

1    14467
0    14033
Name: target, dtype: int64

In [234]:
mana_df.iloc[28500:57000]['target'].value_counts()

1    14736
0    13764
Name: target, dtype: int64

In [235]:
mana_df.iloc[57000:]['target'].value_counts()

1    3607
0    3481
Name: target, dtype: int64

In [221]:
df["No Finding"].value_counts()

-2.0    201033
 1.0     22381
Name: No Finding, dtype: int64

In [143]:
mana_df["target"].value_counts()

1    22895
0    22381
Name: target, dtype: int64

In [139]:
sum=0
for i in mana_df["label"].value_counts():
    sum+=i
sum

45276

In [90]:
sum=0
for i in x:
    if(i>100):
        sum+=100
    else:
        sum+=i
sum

22895

In [19]:
df["label"].nunique()

903

In [20]:
df.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,label
0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,0.0,-2.0,-2.0,-2.0,1.0,
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,-2.0,-2.0,-1.0,1.0,-2.0,-1.0,-1.0,-2.0,-1.0,-2.0,-1.0,-2.0,1.0,-2.0,Lung OpacityFracture
2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,-2.0,-2.0,-2.0,1.0,-2.0,-2.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,1.0,-2.0,Lung OpacityFracture
3,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Lateral,-2,-2.0,-2.0,-2.0,1.0,-2.0,-2.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,1.0,-2.0,Lung OpacityFracture
4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,-2.0,-2.0,-2.0,-2.0,-2.0,1.0,-2.0,-2.0,-2.0,0.0,-2.0,-2.0,-2.0,-2.0,Edema


In [53]:
target = torch.randint(0, 10, (10,))
list = torch.nn.functional.one_hot(target)

In [54]:
list

tensor([[0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

In [3]:
!nvidia-smi

Wed Nov 10 09:18:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 440.82       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0 Off |                  N/A |
| 20%   23C    P8     8W / 250W |     10MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:06:00.0 Off |                  N/A |
| 20%   22C    P8     6W / 250W |   7719MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G